# Example 14.2 Joule-Thomson Effect

A stream of nitrogen is expanded from T1 = 300 K, P1 = 200 bar, to 1 bar by a throttling valve. An ideal throttling valve has the conditions of being adiabatic (no heat loss, energy is conserved); and is either solved using a valve Cv to solve for pressure or solved with the outlet pressure directly specified. 

Calculate the outlet temperature using:

(1) A high precision (helmholtz fundamental) equation of state

(2) The Peng-Robinson equation of state

In [1]:
# Set the conditions and imports
from scipy.constants import bar
from thermo import ChemicalConstantsPackage, PRMIX, CEOSLiquid, CoolPropLiquid, CEOSGas, CoolPropGas, FlashPureVLS
fluid = 'nitrogen'
constants, correlations = ChemicalConstantsPackage.from_IDs([fluid])

T1 = 300.0
P1 = 200*bar
P2 = 1*bar
zs = [1]

In [2]:
# Thermo can use CoolProp to provide properties of one or all phases
# For pure species this is quite reliable within the temperature,
# pressure, etc. limits of the EOSs implemented by CoolProp 

backend = 'HEOS'
gas = CoolPropGas(backend, fluid, T=T1, P=P1, zs=zs)
liquid = CoolPropLiquid(backend, fluid, T=T1, P=P1, zs=zs)

flasher = FlashPureVLS(constants, correlations, gas=gas, liquids=[liquid], solids=[])

state_1 = flasher.flash(T=T1, P=P1)
state_2 = flasher.flash(H=state_1.H(), P=P2)
T2_precise = state_2.T
T2_precise

269.1866854380218

In [3]:
# Use the default originally published Peng-Robinson models
eos_kwargs = dict(Tcs=constants.Tcs, Pcs=constants.Pcs, omegas=constants.omegas)
liquid = CEOSLiquid(PRMIX, HeatCapacityGases=correlations.HeatCapacityGases, eos_kwargs=eos_kwargs)
gas = CEOSGas(PRMIX, HeatCapacityGases=correlations.HeatCapacityGases, eos_kwargs=eos_kwargs)
flasher = FlashPureVLS(constants, correlations, gas=gas, liquids=[liquid], solids=[])

state_1 = flasher.flash(T=T1, P=P1)
state_2 = flasher.flash(H=state_1.H(), P=P2)
T2_PR = state_2.T
T2_PR

265.400194181466

The outlet temperature anwsers given in the book are 269.19 K for the high-precision EOS, and for the PR EOS they used a very low precision Cp of 1 J/(g*K) and obtained an outlet temperature of 283.05 K.

The book textbook cites this 14 K difference as coming from the cubic EOS's lack of precision but the above calculation shows that if an accurate heat capacity is used the difference is only ~ 4K.